# Fine-tuning BERT model for NLP-Challenge (28-Class problem)




In [ ]:
pip install transformers

     |████████████████████████████████| 1.3MB 8.7MB/s 
     |████████████████████████████████| 2.9MB 36.6MB/s 
     |████████████████████████████████| 890kB 47.5MB/s 
     |████████████████████████████████| 1.1MB 48.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=eed77a2f64b8a7ecf513b24cd662c9eb9d427316eedf43b4c664bdeac3088444
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%ls "/content/gdrive/My Drive"

Mounted at /content/gdrive
'Colab Notebooks'/   template_submissions.csv   train.json
 DistilBERT.csv      test_distilber_fine.csv    train_label.csv
 metrics/            test.json
 saved_models/       test_roberta_fine.csv


In [ ]:
import sys
sys.path.append("/content/gdrive/My Drive/")
from transformers import RobertaTokenizer
from transformers import TFRobertaForSequenceClassification, TFRobertaModel
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow.keras as keras
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.0


In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.100.4.154:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.100.4.154:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.100.4.154:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# The data

In [ ]:
PATH = '/content/gdrive/My Drive/'
df = pd.read_json(PATH+'train.json').set_index('Id').loc[:, 'description']
labels = pd.read_csv(PATH+'train_label.csv', index_col=0).loc[:, 'Category'].astype('category').cat.codes

In [ ]:
data_text, data_label = df.tolist(), labels.tolist()
trainval_texts, test_texts, trainval_labels, test_labels = train_test_split(data_text, data_label, 
                                                                            test_size=1/6, 
                                                                            stratify=data_label, 
                                                                            random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(trainval_texts, trainval_labels, 
                                                                    test_size=1/5, 
                                                                    stratify=trainval_labels, 
                                                                    random_state=42)

In [ ]:
len(test_labels)

36200

In [ ]:
weights = np.log(compute_class_weight('balanced', np.unique(trainval_labels), trainval_labels))
weights[weights < 1] = 1

class_weights = dict(enumerate(weights))
class_weights

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels))

In [ ]:
def create_model():
  model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=28)
  model.layers[0].trainable = False
  model.compile(optimizer="adam", 
              loss=model.compute_loss, 
              metrics=['accuracy'])
  return model

In [ ]:
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model = create_model()
model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124645632 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  612124    
Total params: 125,257,756
Trainable params: 612,124
Non-trainable params: 124,645,632
_________________________________________________________________


In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
model.fit(train_dataset.batch(32),
          epochs=20,
          validation_data=val_dataset.batch(32),
          callbacks=callbacks)

Epoch 1/20
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/4525 [..............................] - ETA: 2:37 - loss: 3.2126 - accuracy: 0.1562WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0079s vs `on_train_batch_end` time: 0.0572s). Check your callbacks.


4525/4525 [==============================] - ETA: 0s - loss: 1.1682 - accuracy: 0.6639WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0037s vs `on_test_batch_end` time: 0.0486s). Check your callbacks.


4525/4525 [==============================] - 366s 81ms/step - loss: 1.1682 - accuracy: 0.6639 - val_loss: 0.8371 - val_accuracy: 0.7365
Epoch 2/20
4525/4525 [==============================] - 348s 77ms/step - loss: 1.0049 - accuracy: 0.7061 - val_loss: 0.7655 - val_accuracy: 0.7615
Epoch 3/20
4525/4525 [==============================] - 347s 77ms/step - loss: 0.9697 - accuracy: 0.7127 - val_loss: 0.7510 - val_accuracy: 0.7648
Epoch 4/20
4525/4525 [==============================] - 347s 77ms/step - loss: 0.9583 - accuracy: 0.7148 - val_loss: 0.7377 - val_accuracy: 0.7688
Epoch 5/20
4525/4525 [==============================] - 347s 77ms/step - loss: 0.9469 - accuracy: 0.7178 - val_loss: 0.7465 - val_accuracy: 0.7648
Epoch 6/20
4525/4525 [==============================] - 350s 77ms/step - loss: 0.9378 - accuracy: 0.7209 - val_loss: 0.7277 - val_accuracy: 0.7744
Epoch 7/20
4525/4525 [==============================] - 352s 78ms/step - loss: 0.9308 - accuracy: 0.7221 - val_loss: 0.7201 - val

In [ ]:
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
    optimizer = tf.keras.optimizers.Adam(learning_rate=1.5e-5)
    model.trainable=True
    model.compile(optimizer=optimizer, 
              loss=model.compute_loss, 
              metrics=['accuracy'])
model.summary()


Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124645632 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  612124    
Total params: 125,257,756
Trainable params: 125,257,756
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)]
model.fit(train_dataset.batch(48),
          epochs=5,
          validation_data=val_dataset.batch(48))

Epoch 1/5


   2/3017 [..............................] - ETA: 16:33:10 - loss: 1.1240 - accuracy: 0.6667WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0098s vs `on_train_batch_end` time: 0.2226s). Check your callbacks.


3017/3017 [==============================] - ETA: 0s - loss: 0.6180 - accuracy: 0.8151WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0040s vs `on_test_batch_end` time: 0.0640s). Check your callbacks.


3017/3017 [==============================] - 848s 281ms/step - loss: 0.6180 - accuracy: 0.8151 - val_loss: 0.5112 - val_accuracy: 0.8463
Epoch 2/5
3017/3017 [==============================] - 757s 251ms/step - loss: 0.4991 - accuracy: 0.8475 - val_loss: 0.5003 - val_accuracy: 0.8527
Epoch 3/5
3017/3017 [==============================] - 757s 251ms/step - loss: 0.4364 - accuracy: 0.8632 - val_loss: 0.5124 - val_accuracy: 0.8519
Epoch 4/5
3017/3017 [==============================] - 757s 251ms/step - loss: 0.3649 - accuracy: 0.8846 - val_loss: 0.5258 - val_accuracy: 0.8524
Epoch 5/5
3017/3017 [==============================] - 758s 251ms/step - loss: 0.3044 - accuracy: 0.9007 - val_loss: 0.5669 - val_accuracy: 0.8513


In [ ]:
save_directory = PATH+"saved_models/roberta_real_fine_tuning"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/gdrive/My Drive/saved_models/roberta_real_fine_tuning/tokenizer_config.json',
 '/content/gdrive/My Drive/saved_models/roberta_real_fine_tuning/special_tokens_map.json',
 '/content/gdrive/My Drive/saved_models/roberta_real_fine_tuning/vocab.json',
 '/content/gdrive/My Drive/saved_models/roberta_real_fine_tuning/merges.txt',
 '/content/gdrive/My Drive/saved_models/roberta_real_fine_tuning/added_tokens.json')

In [ ]:
save_directory = PATH+"saved_models/roberta_real_fine_tuning"
loaded_tokenizer = RobertaTokenizer.from_pretrained(save_directory)
loaded_model = TFRobertaForSequenceClassification.from_pretrained(save_directory)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at /content/gdrive/My Drive/saved_models/roberta_real_fine_tuning.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
y_true = pd.Series(test_labels, name='y_true')
y = []
for i, text in enumerate(test_texts):
    predict_input = loaded_tokenizer.encode(text,truncation=True,padding=True,return_tensors="tf")
    output = loaded_model(predict_input)[0]
    y.append(tf.argmax(output, axis=1).numpy()[0])
    print(i,"/ 36200",round(i*100/36200,2))
y_pred = pd.Series(y, name='y_pred')
lala = pd.concat([y_pred,y_true], axis=1)
lala.to_csv(PATH+"test_roberta_fine.csv")

Streaming output truncated to the last 5000 lines.
31200 / 36200 86.19
31201 / 36200 86.19
31202 / 36200 86.19
31203 / 36200 86.2
31204 / 36200 86.2
31205 / 36200 86.2
31206 / 36200 86.2
31207 / 36200 86.21
31208 / 36200 86.21
31209 / 36200 86.21
31210 / 36200 86.22
31211 / 36200 86.22
31212 / 36200 86.22
31213 / 36200 86.22
31214 / 36200 86.23
31215 / 36200 86.23
31216 / 36200 86.23
31217 / 36200 86.23
31218 / 36200 86.24
31219 / 36200 86.24
31220 / 36200 86.24
31221 / 36200 86.25
31222 / 36200 86.25
31223 / 36200 86.25
31224 / 36200 86.25
31225 / 36200 86.26
31226 / 36200 86.26
31227 / 36200 86.26
31228 / 36200 86.27
31229 / 36200 86.27
31230 / 36200 86.27
31231 / 36200 86.27
31232 / 36200 86.28
31233 / 36200 86.28
31234 / 36200 86.28
31235 / 36200 86.28
31236 / 36200 86.29
31237 / 36200 86.29
31238 / 36200 86.29
31239 / 36200 86.3
31240 / 36200 86.3
31241 / 36200 86.3
31242 / 36200 86.3
31243 / 36200 86.31
31244 / 36200 86.31
31245 / 36200 86.31
31246 / 36200 86.31
31247 / 36200 86.

In [ ]:
from sklearn.metrics import classification_report # DistilBERT PROPREMENT
print(classification_report(test_labels, y))

              precision    recall  f1-score   support

           0       0.67      0.66      0.67       250
           1       0.87      0.81      0.84       686
           2       0.84      0.72      0.77       157
           3       0.70      0.63      0.66      1524
           4       0.84      0.77      0.80       135
           5       0.88      0.77      0.82       770
           6       0.78      0.82      0.80      2049
           7       0.77      0.71      0.74       143
           8       0.81      0.82      0.82      1103
           9       0.82      0.80      0.81       520
          10       0.83      0.74      0.78       139
          11       0.80      0.74      0.77      1935
          12       0.91      0.75      0.83       273
          13       0.79      0.73      0.76       677
          14       0.88      0.87      0.87      2104
          15       0.84      0.73      0.78       715
          16       0.93      0.95      0.94       908
          17       0.77    

In [ ]:
from sklearn.metrics import classification_report

# -*- coding: utf-8 -*-
"""
Created on Sat Oct 17 11:41:09 2020

@author: dylan_monfret
"""

# pd.set_option("display.max_rows", None, "display.max_columns", None)

" Local Function "


def di_calculation(genres_dict):
    """
    Function to calculate Disparate Impact.

    :param genres_dict: Dictionary of genre accounting by label
    :return: the Disparate Impact mesure, a measure of the imbalance between 2 sub-group in a given group (here 'M' and
    'F' for a specific job label).
    """

    if not genres_dict:
        return 0

    try:  # Disparate Impact definition.
        m_count = genres_dict["M"]
        f_count = genres_dict["F"]
        the_di = max(m_count, f_count) / min(m_count, f_count)
        return the_di

    except KeyError:  # Could happen essentially if a group is "empty".

        if "M" not in genres_dict.keys():
            return genres_dict["F"]

        elif "F" not in genres_dict.keys():
            return genres_dict["M"]


def report_dict_creator(sklearn_report, true_values, pred_values, genre_values, a_label_list):
    """
    A function to create enhanced report including DI and macro averages.

    :param sklearn_report: basic sklearn.classification_report dictionary.
    :param true_values: y_true
    :param pred_values: y_pred
    :param genre_values: vector of genres
    :param a_label_list: list of possible value taken by y_true / y_pred
    :return: the enhanced dictionary report including DIs and other macro averages
    """
    # macros DI init
    true_di_sum = float()
    pred_di_sum = float()
    diff_di_sum = float()
    the_len = len(a_label_list)

    for label in a_label_list:
        # Count 'M' / 'F' apparition for a specific label
        # Label filtering made with numpy.where
        unique1, counts1 = np.unique(genre_values[np.where(true_values == label)], return_counts=True)
        unique2, counts2 = np.unique(genre_values[np.where(pred_values == label)], return_counts=True)
        true_dict_di = dict(zip(unique1, counts1))
        pred_dict_di = dict(zip(unique2, counts2))

        # 'di_calculation' call and DI difference calculation
        true_di = di_calculation(true_dict_di)
        pred_di = di_calculation(pred_dict_di)
        diff_di = abs(true_di - pred_di)

        # Results added to former classification_report dictionary in specific label section
        sklearn_report[label]["true_di"] = true_di
        sklearn_report[label]["pred_di"] = pred_di
        sklearn_report[label]["diff_di"] = diff_di

        # DIs summed for DIs macros calculation
        true_di_sum += true_di
        pred_di_sum += pred_di
        diff_di_sum += diff_di

    # DIs macros added to former classification_report dictionary in 'macro avg' section
    sklearn_report["macro avg"]["true_di"] = true_di_sum / the_len
    sklearn_report["macro avg"]["pred_di"] = pred_di_sum / the_len
    sklearn_report["macro avg"]["diff_di"] = diff_di_sum / the_len

    return sklearn_report


def report_df_creator(enhanced_report):
    """
    A function to transform classification report dictionary into a clean pandas.DataFrame.

    :param enhanced_report: classification report dictionary with DIs.
    :return: classification report df sorted by index.
    """

    # Change the dictionary into a pandas.DataFrame
    report_df = pd.DataFrame(enhanced_report).transpose()

    # Sort by index with labels first (need addition then deletion of leading zeros)
    # For our problem, only 1 leading zero by leading are necessary
    report_df.index = report_df.index.map(lambda x: '{0:0>2}'.format(x))
    report_df = report_df.sort_index()
    report_df.index = report_df.index.map(lambda x: '{}'.format(x.lstrip("0")))

    # Filling of accuracy line with 'blank' cell
    report_df.loc["accuracy"] = [np.NaN, np.NaN, enhanced_report["accuracy"], np.NaN, np.NaN, np.NaN, np.NaN]

    # Addition of some weighted averages (DIs and DIs differences)
    report_df.loc["weighted avg", "true_di"] = (
            report_df.true_di[:-3] * report_df.support[:-3] / report_df.support[:-3].sum()).sum()
    report_df.loc["weighted avg", "pred_di"] = (
            report_df.pred_di[:-3] * report_df.support[:-3] / report_df.support[:-3].sum()).sum()
    report_df.loc["weighted avg", "diff_di"] = (
            report_df.diff_di[:-3] * report_df.support[:-3] / report_df.support[:-3].sum()).sum()

    return report_df


" Class definition "


class Report:
    def __init__(self, array_result):
        """
        'Report' is a classification_report dictionary type from 'sklearn.metrics', but which include the Disparate
        Impact for each class / label.

        :param array_result: numpy.array from a machine learning process with 3 distinct elements: the first containing
        real test values, the second containing predicted values and the last representing the sensitive genre variable
        of modality 'F' or 'M'. For a correct method's execution, array's elements must be kept in order.
        """

        # Extraction
        true = array_result[0]
        pred = array_result[1]
        genres = array_result[2]
        label_list = list(set(true).union(set(pred)))

        # Basic classification_report to enhance with DIs
        basic_report = classification_report(true, pred, labels=label_list, output_dict=True, zero_division='warn')

        # Report dictionary created with DI calculated
        self.report_dict = report_dict_creator(basic_report, true, pred, genres, label_list)

        # Report dataframe created with previous dictionary
        self.report_df = report_df_creator(self.report_dict)

    def __repr__(self):
        """
        :return: The report's pandas.DataFrame (self.report_df) as a string.
        """
        return self.report_df.to_string()

In [ ]:
genre = pd.read_json(PATH+'train.json').set_index('Id').loc[:, 'gender'].astype('category').cat.codes
genre.to_numpy()

array([0, 1, 1, ..., 1, 0, 0], dtype=int8)

In [ ]:
arr = np.array([test_labels, y, genre])
Report(arr)

ValueError: ignored